In [11]:
import os, getpass
from ibm_watsonx_ai import Credentials, APIClient
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM
from langchain.chains import RetrievalQA

In [12]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="VPryyL4t508YQ_pJQNCU0m23Yctctahrqo2pjT5BMQkV"
)
project_id = "6552c3bb-a35a-4bf3-8430-c42828d59a06"
api_client = APIClient(credentials=credentials, project_id=project_id)

In [13]:
# filename = "data/email.csv"
# loader = TextLoader(filename)
# documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)
# texts